# Audiobooks business case

### Extract the data from the csv

In [1]:
import tensorflow as tf

In [2]:
import numpy as np 
from sklearn import preprocessing 

raw_csv_data = np.loadtxt('Audiobooks_data.csv' , delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []


for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)


unscaled_input_equal_priors = np.delete(unscaled_inputs_all , indices_to_remove , axis= 0)
targets_equal_priors =np.delete(targets_all , indices_to_remove , axis= 0)   

### Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_input_equal_priors)

### Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [6]:
sample_count = shuffled_inputs.shape[0]

train_sample_count = int(0.8*sample_count)
validation_sample_count = int(0.1*sample_count )
test_sample_count = sample_count - train_sample_count - validation_sample_count 

train_inputs = shuffled_inputs[:train_sample_count]
train_targets = shuffled_targets[:train_sample_count]

validation_inputs = shuffled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets = shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

test_inputs = shuffled_inputs[train_sample_count+validation_sample_count:]
test_targets = shuffled_targets[train_sample_count+validation_sample_count:]

print(np.sum(train_targets) , train_sample_count , np.sum(train_targets)/train_sample_count)
print(np.sum(validation_targets), validation_sample_count, np.sum(validation_targets) / validation_sample_count)
print(np.sum(test_targets), test_sample_count, np.sum(test_targets) / test_sample_count)


1783.0 3579 0.4981838502374965
232.0 447 0.5190156599552572
222.0 448 0.4955357142857143


### Save the three datasets in *.npz

In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

# Create the machine learning algorithm


# Import the relevant libraries

In [8]:
import numpy as np
import tensorflow as tf

In [15]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float32)
train_targets = npz['targets'].astype(np.int64)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float32), npz['targets'].astype(np.int64)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float32), npz['targets'].astype(np.int64)

# Model

In [10]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

In [11]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
batch_size = 100
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size,
          epochs=max_epochs, 
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose = 2 
          )  

Epoch 1/100
36/36 - 1s - loss: 0.6008 - accuracy: 0.6519 - val_loss: 0.5230 - val_accuracy: 0.7248 - 878ms/epoch - 24ms/step
Epoch 2/100
36/36 - 0s - loss: 0.4713 - accuracy: 0.7605 - val_loss: 0.4492 - val_accuracy: 0.7785 - 78ms/epoch - 2ms/step
Epoch 3/100
36/36 - 0s - loss: 0.4173 - accuracy: 0.7865 - val_loss: 0.4128 - val_accuracy: 0.7942 - 78ms/epoch - 2ms/step
Epoch 4/100
36/36 - 0s - loss: 0.3892 - accuracy: 0.7963 - val_loss: 0.3951 - val_accuracy: 0.8009 - 83ms/epoch - 2ms/step
Epoch 5/100
36/36 - 0s - loss: 0.3732 - accuracy: 0.8013 - val_loss: 0.3838 - val_accuracy: 0.8143 - 77ms/epoch - 2ms/step
Epoch 6/100
36/36 - 0s - loss: 0.3625 - accuracy: 0.8064 - val_loss: 0.3805 - val_accuracy: 0.7785 - 78ms/epoch - 2ms/step
Epoch 7/100
36/36 - 0s - loss: 0.3556 - accuracy: 0.8064 - val_loss: 0.3713 - val_accuracy: 0.8031 - 82ms/epoch - 2ms/step
Epoch 8/100
36/36 - 0s - loss: 0.3495 - accuracy: 0.8080 - val_loss: 0.3685 - val_accuracy: 0.8233 - 105ms/epoch - 3ms/step
Epoch 9/100
3

# Test the model

In [16]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3233 - accuracy: 0.8482


In [17]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.32. Test accuracy: 84.82%
